## The Battle of Neighborhoods 邻里之争

#### Author: Hu Junjie

#### 作者：胡竣杰

## 1. Introduction 介绍

**Zhongshan** is my hometown. It locates at the west side of the Pearl Delta and it is famous for the birth of Sun Yet-Sun. This great man in the modern history of China is the reason why this city changed its name into 'Zhongshan' from 'Heung Shan'. Before the policy of reformation and opening, Zhongshan used to be much more prosperous than Shenzhen as a historical city. However, in the recent decades, Zhongshan is lossing its glory in tourism and the share of Economy. 
Let's find out what's the current status of tourist attraction for Zhongshan.

中山是我的家乡。它位于珠江三角洲的西侧，闻名于孙中山先生的诞生。这位中国近代史上的伟人是这座城市从“香山”更名为“中山”的原因。改革开放前，中山作为一个历史名城，比深圳繁荣得多。然而，近几十年来，中山在旅游业和经济中所占的份额正在逐渐丧失。
让我们一起来了解一下中山目前的旅游景点情况。

Based on data science, we try to separate each district of Zhongshan through machine learning and tie the districts together to present a new perspective for understanding Zhongshan to those who visit this place for the first time.
First of all, using geocoder, we will secure and visualize information in each district of Zhongshan. The Foursquare API will allow us to explore multiple venues in each district. After sorting it into Pandas dataframe through hot-end coding and normalization, Zhongshan will be divided into about five zones with similar characteristics to provide tourists with rough local information.

在数据科学的基础上，我们尝试通过机器学习将中山的各个区域分开，并将这些区域联系在一起，为第一次来这里的人提供一个了解中山的新视角。
首先，我们使用geocoder对中山各区的信息进行加密和可视化。Foursquare API将允许我们在每个区域探索多个场所。通过热端编码和归一化，将其整理成Pandas Dataframe，将中山划分为五个特征相似的区域，为游客建议最合适的目的地。

## 2. Data source and Preprocessing Data 

We try to collect data in the similar way, referring to the method we did in the previous example - Segmenting and Clustering Neighborhoods in New York City.
* District of Zhongshan : https://www.atool99.com/china_city.php
* Folium, Geocoder
    * These will be used to visualize map info.

In [2]:
import requests # library to handle requests
from bs4 import BeautifulSoup  # import beautiful soup for html parsing

import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

#!conda install -c conda-forge geopy --yes  # uncomment if geopy library is not installed
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

#!conda install -c conda-forge folium=0.5.0 --yes   # uncomment if folium library is not installed
import folium # map rendering library

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

print('Libraries imported.')

Libraries imported.


In [3]:
url = 'https://www.atool99.com/city_442000.html'

In [4]:
df = pd.read_html(url,header=0,encoding='utf8')[1]

In [5]:
df

,区位码,地区名称,电话区号,邮政编码,经纬度,英文
0,442001,石岐区,760,528400,"(113.378835, 22.52522)",Shiqi
1,442004,南区,760,528400,"(113.355896, 22.486568)",Nanqu
2,442005,五桂山区,760,528458,"(113.41079, 22.51968)",Wuguishan
3,442006,火炬开发区,760,528437,"(113.480523, 22.566082)",Huoju
4,442007,黄圃镇,760,528429,"(113.342359, 22.715116)",Huangpu
5,442008,南头镇,760,528421,"(113.296358, 22.713907)",Nantou
6,442009,东凤镇,760,528425,"(113.26114, 22.68775)",Dongfeng
7,442010,阜沙镇,760,528434,"(113.353024, 22.666364)",Fusha
8,442011,小榄镇,760,528415,"(113.244235, 22.666951)",Xiaolan
9,442012,东升镇,760,528400,"(113.296298, 22.614003)",Dongsheng


In [6]:
df_new = pd.DataFrame()

#put Name in first
df_new['District']=df.英文

#split ‘经纬度’ into 'Latitudes' & 'Longitudes'
df_new['Latitude'] = df.经纬度.str.split(',').str[1]
df_new['Longitude'] = df.经纬度.str.split(',').str[0]
df_new.head()

,District,Latitude,Longitude
0,Shiqi,22.52522),(113.378835
1,Nanqu,22.486568),(113.355896
2,Wuguishan,22.51968),(113.41079
3,Huoju,22.566082),(113.480523
4,Huangpu,22.715116),(113.342359


In [7]:
# cleaning
df_new['Latitude']=df_new.Latitude.str.split(')').str[0]
df_new['Longitude']=df_new.Longitude.str.split('(').str[1]
df_new.head()

,District,Latitude,Longitude
0,Shiqi,22.52522,113.378835
1,Nanqu,22.486568,113.355896
2,Wuguishan,22.51968,113.41079
3,Huoju,22.566082,113.480523
4,Huangpu,22.715116,113.342359


## 3. Explore and Cluster the Neighborhoods in Zhongshan

Use geopy library to get the latitude and longitude values of Zhongshan.¶

In [8]:
address = "Zhongshan, ZS"

geolocator = Nominatim(user_agent="zhongshan_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Zhongshan city are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Zhongshan city are 22.5213807, 113.3656141.


Create a map of Zhongshan with neighborhoods superimposed on top.

In [9]:
# create map of Toronto using latitude and longitude values
map_zs = folium.Map(location=[latitude, longitude], zoom_start=10)
map_zs

add markers to the map

In [10]:
for Latitude, Longitude, District in zip(
        df_new['Latitude'], 
        df_new['Longitude'], 
        df_new['District']):
    label = '{}'.format(District)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [Latitude, Longitude],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_zs)  

map_zs

Define Foursquare Credentials and Version

In [11]:
CLIENT_ID = 'F5LKCXZIMXVSFZJNXKRC00RHRT4440ESDKNCBE1SWQZNZLPS' # your Foursquare ID
CLIENT_SECRET = 'IX0DMTBNNBT5NIF421DGULYQ4MX0ICMMM5M3AQASO5HSYVJQ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: F5LKCXZIMXVSFZJNXKRC00RHRT4440ESDKNCBE1SWQZNZLPS
CLIENT_SECRET:IX0DMTBNNBT5NIF421DGULYQ4MX0ICMMM5M3AQASO5HSYVJQ


In [12]:
#Explore the first District in our data frame "df_new"
district_name = df_new.loc[0, 'District']
print(f"The first District's name is '{district_name}'.")

The first District's name is 'Shiqi'.


In [13]:
district_latitude = df_new.loc[0, 'Latitude'] # neighborhood latitude value
district_longitude = df_new.loc[0, 'Longitude'] # neighborhood longitude value

print('Latitude and longitude values of {} are {}, {}.'.format(district_name, 
                                                               district_latitude, 
                                                               district_longitude))

Latitude and longitude values of Shiqi are  22.52522, 113.378835.


Now, let's get the top 100 venues that are in The Shiqi within a radius of 5000 meters.

In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius=5000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['District', 
                  'District Latitude', 
                  'District Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [15]:
zs_venues = getNearbyVenues(names=df_new['District'],
                                   latitudes=df_new['Latitude'],
                                   longitudes=df_new['Longitude']
                                  )

print('................')
print('Process is done.')

Shiqi
Nanqu
Wuguishan
Huoju
Huangpu
Nantou
Dongfeng
Fusha
Xiaolan
Dongsheng
Guzhen
Henglan
Sanjiao
Minzhong
Nanlang
Gangkou
Dayong
Shaxi
Sanxiang
Banfu
Shenwan
Tanzhou
................
Process is done.


In [16]:
#Let's check the size of the resulting dataframe
print(zs_venues.shape)
zs_venues.head()

(226, 7)


,District,District Latitude,District Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Shiqi,22.52522,113.378835,Hilton,22.515790,113.377958,Hotel
1,Shiqi,22.52522,113.378835,King Century Hotel (京华世纪酒店),22.513871,113.379753,Motel
2,Shiqi,22.52522,113.378835,Lihe Plaza (利和购物中心),22.514559,113.378863,Shopping Mall
3,Shiqi,22.52522,113.378835,Sheraton Zhongshan Hotel (中山喜来登酒店),22.519587,113.365065,Hotel
4,Shiqi,22.52522,113.378835,Starbucks (星巴克),22.522906,113.385148,Coffee Shop


In [17]:
zs_venues.groupby('District').count()

,District Latitude,District Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
District,,,,,,
Banfu,7,7,7,7,7,7
Dayong,6,6,6,6,6,6
Dongfeng,6,6,6,6,6,6
Dongsheng,5,5,5,5,5,5
Fusha,5,5,5,5,5,5
Gangkou,27,27,27,27,27,27
Guzhen,14,14,14,14,14,14
Henglan,5,5,5,5,5,5
Huangpu,6,6,6,6,6,6


In [18]:
print('There are {} uniques categories.'.format(len(zs_venues['Venue Category'].unique())))

There are 47 uniques categories.


## 4. Analyze Each District

In [19]:
# one hot encoding
zs_onehot = pd.get_dummies(zs_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
zs_onehot['District'] = zs_venues['District'] 

# move neighborhood column to the first column
fixed_columns = [zs_onehot.columns[-1]] + list(zs_onehot.columns[:-1])
zs_onehot = zs_onehot[fixed_columns]

zs_onehot.head()

,District,Art Gallery,Athletics & Sports,Bakery,Boat or Ferry,Bridal Shop,Bus Station,Cafeteria,Cantonese Restaurant,Chinese Restaurant,Coffee Shop,Comfort Food Restaurant,Convenience Store,Department Store,Dumpling Restaurant,Fabric Shop,Fast Food Restaurant,Food Court,Fried Chicken Joint,Garden,German Restaurant,Golf Course,Gym,Harbor / Marina,Hostel,Hot Spring,Hotel,Hotel Pool,Italian Restaurant,Japanese Restaurant,Light Rail Station,Market,Motel,Mountain,Multiplex,Park,Pier,Pizza Place,Ramen Restaurant,Rest Area,Restaurant,Seafood Restaurant,Shopping Mall,Spa,Steakhouse,Tourist Information Center,Train Station,Tunnel
0,Shiqi,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Shiqi,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Shiqi,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,Shiqi,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Shiqi,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [20]:
#And let's examine the new dataframe size.
zs_onehot.shape

(226, 48)

In [21]:
#Next, let's group rows by neighborhood and by taking the mean of the 
#  frequency of occurrence of each category
zs_grouped = zs_onehot.groupby('District').mean().reset_index()
zs_grouped

,District,Art Gallery,Athletics & Sports,Bakery,Boat or Ferry,Bridal Shop,Bus Station,Cafeteria,Cantonese Restaurant,Chinese Restaurant,Coffee Shop,Comfort Food Restaurant,Convenience Store,Department Store,Dumpling Restaurant,Fabric Shop,Fast Food Restaurant,Food Court,Fried Chicken Joint,Garden,German Restaurant,Golf Course,Gym,Harbor / Marina,Hostel,Hot Spring,Hotel,Hotel Pool,Italian Restaurant,Japanese Restaurant,Light Rail Station,Market,Motel,Mountain,Multiplex,Park,Pier,Pizza Place,Ramen Restaurant,Rest Area,Restaurant,Seafood Restaurant,Shopping Mall,Spa,Steakhouse,Tourist Information Center,Train Station,Tunnel
0,Banfu,0.142857,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.142857,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.142857,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.142857,0.0,0.000000,0.0,0.000000,0.142857,0.000000,0.142857,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.142857
1,Dayong,0.166667,0.000000,0.0,0.0,0.166667,0.000000,0.0,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.166667,0.000000,0.0,0.000000,0.166667,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Dongfeng,0.000000,0.000000,0.0,0.0,0.000000,0.166667,0.0,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.166667,0.0,0.166667,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.166667,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000
3,Dongsheng,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.200000,0.200000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.200000,0.2,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.200000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Fusha,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.2,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.2,0.00,0.200000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.20,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,Gangkou,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.074074,0.000000,0.222222,0.000000,0.000000,0.000000,0.0,0.000000,0.185185,0.0,0.000000,0.000000,0.000000,0.000000,0.037037,0.0,0.0,0.00,0.185185,0.0,0.000000,0.0,0.000000,0.000000,0.037037,0.000000,0.000000,0.037037,0.0,0.037037,0.000000,0.000000,0.00,0.000000,0.111111,0.000000,0.000000,0.037037,0.037037,0.000000
6,Guzhen,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.214286,0.000000,0.000000,0.071429,0.0,0.000000,0.071429,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.357143,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.071429,0.000000,0.00,0.000000,0.142857,0.000000,0.000000,0.000000,0.071429,0.000000
7,Henglan,0.000000,0.200000,0.0,0.0,0.200000,0.000000,0.0,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.400000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Huangpu,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.333333,0.0,0.000000,0.0,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.166667,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000

In [22]:
zs_grouped.shape

(22, 48)

Let's print each neighborhood along with the top 10 most common venues

In [23]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['District']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
District_venues_sorted = pd.DataFrame(columns=columns)
District_venues_sorted['District'] = zs_grouped['District']

for ind in np.arange(zs_grouped.shape[0]):
    District_venues_sorted.iloc[ind, 1:] = return_most_common_venues(zs_grouped.iloc[ind, :], num_top_venues)

District_venues_sorted.head()

,District,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Banfu,Tunnel,Mountain,Cantonese Restaurant,Garden,Hotel,Market,Art Gallery,Restaurant,Rest Area,Fried Chicken Joint
1,Dayong,Art Gallery,Garden,Bridal Shop,Cantonese Restaurant,Fabric Shop,Mountain,Department Store,Golf Course,German Restaurant,Fried Chicken Joint
2,Dongfeng,Train Station,Hotel,Bus Station,Italian Restaurant,Chinese Restaurant,Pizza Place,Tunnel,Department Store,Garden,Fried Chicken Joint
3,Dongsheng,Hotel,Food Court,Cantonese Restaurant,Chinese Restaurant,Fast Food Restaurant,Tunnel,Department Store,Golf Course,German Restaurant,Garden
4,Fusha,Hostel,Hotel,Restaurant,Cafeteria,Cantonese Restaurant,Department Store,German Restaurant,Garden,Fried Chicken Joint,Food Court


## 5. Cluster Neighborhoods

In [24]:
# set number of clusters
kclusters = 5

zs_grouped_clustering = zs_grouped.drop('District', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(zs_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 3, 0, 2, 2, 2, 2, 4, 2, 2], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [25]:
# add clustering labels
District_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)


zs_merged = df_new

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
zs_merged = zs_merged.join(District_venues_sorted.set_index('District'), on='District')

zs_merged.head() # check the last columns!

,District,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Shiqi,22.52522,113.378835,2,Coffee Shop,Hotel,Fast Food Restaurant,Shopping Mall,Cantonese Restaurant,Park,Tourist Information Center,Gym,Motel,Train Station
1,Nanqu,22.486568,113.355896,2,Fast Food Restaurant,Coffee Shop,Hotel,Park,Multiplex,Tourist Information Center,Shopping Mall,Chinese Restaurant,Motel,Comfort Food Restaurant
2,Wuguishan,22.51968,113.41079,2,Hotel,Cantonese Restaurant,Fast Food Restaurant,Coffee Shop,Shopping Mall,Spa,Seafood Restaurant,Golf Course,Gym,Pizza Place
3,Huoju,22.566082,113.480523,2,Fast Food Restaurant,Chinese Restaurant,Bakery,Boat or Ferry,Hotel Pool,Pizza Place,Pier,Tunnel,Department Store,German Restaurant
4,Huangpu,22.715116,113.342359,2,Hotel,Spa,Seafood Restaurant,Light Rail Station,Chinese Restaurant,Tunnel,Convenience Store,Garden,Fried Chicken Joint,Food Court


In [26]:
# create map
map_clusters = folium.Map(location=[Latitude, Longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []

for lat, lon, poi, cluster in zip(zs_merged['Latitude'], zs_merged['Longitude'], zs_merged['District'], zs_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 6. Examine Clusters

In [27]:
# Cluster 1
zs_merged.loc[zs_merged['Cluster Labels'] == 0,
                  zs_merged.columns[[0] + 
                  list(range(1, zs_merged.shape[1]))]]

,District,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Nantou,22.713907,113.296358,0,Train Station,Hotel,Seafood Restaurant,Bus Station,Tunnel,Department Store,German Restaurant,Garden,Fried Chicken Joint,Food Court
6,Dongfeng,22.68775,113.26114,0,Train Station,Hotel,Bus Station,Italian Restaurant,Chinese Restaurant,Pizza Place,Tunnel,Department Store,Garden,Fried Chicken Joint
8,Xiaolan,22.666951,113.244235,0,Train Station,Hotel,Bus Station,Italian Restaurant,Chinese Restaurant,Pizza Place,Tunnel,Department Store,Garden,Fried Chicken Joint


In [28]:
# Cluster 2
zs_merged.loc[zs_merged['Cluster Labels'] == 1,
                zs_merged.columns[[0] + 
                  list(range(1, zs_merged.shape[1]))]]

,District,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Sanjiao,22.677033,113.423624,1,Golf Course,Hotel,Fast Food Restaurant,Rest Area,Dumpling Restaurant,Convenience Store,German Restaurant,Garden,Fried Chicken Joint,Food Court
18,Sanxiang,22.352494,113.4334,1,Golf Course,Steakhouse,Hot Spring,Hotel,Convenience Store,German Restaurant,Garden,Fried Chicken Joint,Food Court,Fast Food Restaurant


In [29]:
# Cluster 3
zs_merged.loc[zs_merged['Cluster Labels'] == 2,
                 zs_merged.columns[[0] + 
                  list(range(1, zs_merged.shape[1]))]]

,District,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Shiqi,22.52522,113.378835,2,Coffee Shop,Hotel,Fast Food Restaurant,Shopping Mall,Cantonese Restaurant,Park,Tourist Information Center,Gym,Motel,Train Station
1,Nanqu,22.486568,113.355896,2,Fast Food Restaurant,Coffee Shop,Hotel,Park,Multiplex,Tourist Information Center,Shopping Mall,Chinese Restaurant,Motel,Comfort Food Restaurant
2,Wuguishan,22.51968,113.41079,2,Hotel,Cantonese Restaurant,Fast Food Restaurant,Coffee Shop,Shopping Mall,Spa,Seafood Restaurant,Golf Course,Gym,Pizza Place
3,Huoju,22.566082,113.480523,2,Fast Food Restaurant,Chinese Restaurant,Bakery,Boat or Ferry,Hotel Pool,Pizza Place,Pier,Tunnel,Department Store,German Restaurant
4,Huangpu,22.715116,113.342359,2,Hotel,Spa,Seafood Restaurant,Light Rail Station,Chinese Restaurant,Tunnel,Convenience Store,Garden,Fried Chicken Joint,Food Court
7,Fusha,22.666364,113.353024,2,Hostel,Hotel,Restaurant,Cafeteria,Cantonese Restaurant,Department Store,German Restaurant,Garden,Fried Chicken Joint,Food Court
9,Dongsheng,22.614003,113.296298,2,Hotel,Food Court,Cantonese Restaurant,Chinese Restaurant,Fast Food Restaurant,Tunnel,Department Store,Golf Course,German Restaurant,Garden
10,Guzhen,22.611019,113.179745,2,Hotel,Coffee Shop,Shopping Mall,Train Station,Fast Food Restaurant,Ramen Restaurant,Department Store,Convenience Store,Garden,Fried Chicken Joint
14,Nanlang,22.492378,113.533939,2,Comfort Food Restaurant,Mountain,Train Station,Hotel,Fried Chicken Joint,Seafood Restaurant,Fast Food Restaurant,Cantonese Restaurant,Chinese Restaurant,Garden
15,Gangkou,22.521113,113.382391,2,Coffee Shop,Hotel,Fast Food Restaurant,Shopping Mall,Cantonese Restaurant,Park,Tourist Information Center,Gym,Motel,Train Station


In [30]:
# Cluster 4
zs_merged.loc[zs_merged['Cluster Labels'] == 3,
                 zs_merged.columns[[0] + 
                  list(range(1, zs_merged.shape[1]))]]

,District,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,Dayong,22.467712,113.291708,3,Art Gallery,Garden,Bridal Shop,Cantonese Restaurant,Fabric Shop,Mountain,Department Store,Golf Course,German Restaurant,Fried Chicken Joint
19,Banfu,22.415674,113.320346,3,Tunnel,Mountain,Cantonese Restaurant,Garden,Hotel,Market,Art Gallery,Restaurant,Rest Area,Fried Chicken Joint


In [31]:
# Cluster 5
zs_merged.loc[zs_merged['Cluster Labels'] == 4,
                 zs_merged.columns[[0] + 
                  list(range(1, zs_merged.shape[1]))]]

,District,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Henglan,22.523202,113.265845,4,Rest Area,Athletics & Sports,Bridal Shop,Chinese Restaurant,Tunnel,Department Store,Golf Course,German Restaurant,Garden,Fried Chicken Joint
13,Minzhong,22.623468,113.486025,4,Chinese Restaurant,Restaurant,Rest Area,Tunnel,Convenience Store,German Restaurant,Garden,Fried Chicken Joint,Food Court,Fast Food Restaurant


## 7. Discussion 讨论

Since Foursquare constantly updates the database, the specific clustering results obtained by running the above code at different times may change, so the results discussed below may not be consistent with the above data. The following link to github is the cluster data which matchs the analysis . Please have a look:

因为Foursquare会不断的更新数据库，所以不同时间运行以上代码后得到的具体分群结果会有改变，所以以下的讨论结果与上面的数据不一定相符合。以下是我第一次跑出来的分群数据得出的分析，请具体看:

https://github.com/Woochunkit/Coursera_Capstone/blob/main/Assignment%204%20%26%205-2.ipynb


Using the most common place information above, we might explain characteristics of each cluster as below:

 1. Cluster #1 has 8 districts and they are quite commercial. You could easily find venues such as hotel, coffee shops, Fastfood restaurants. It would be good for tourist to enjoy it and have nice experience here. 

2. Cluster #2 has 4 districts and they functions quite much as transportation junction. You could easily find Light Rail Station, Bus Station and Hotel.

3. Cluster #3 has 1 district. This district, Henglan, is not commercial but good place for rest area. 

4. Cluster #4 has 4 districts and they are not quite commercial but living towns. You could easily find Fast food, Chinese food, Cantonese food.

5. Cluster #5 has 5 districts and they are good for tourism. You can easily find Hotel, garden, restaurants and some special venue.

使用上面最常见的位置信息，我们可以解释每个簇的特征如下:
1. 1号簇有8个镇/区，石岐、南区、五桂山、古镇、三角、港口、沙溪、坦洲，它们都很商业化。你可以很容易地找到诸如酒店、咖啡店、快餐店等场所。对游客来说，在这里享受和体验会很好。
2. 2号簇有4个镇/区，黄埔、南头、东凤、小榄，它们的功能非常重要，是交通枢纽。你可以很容易地找到轻轨站，汽车站和酒店。
3. 3号簇有1个镇/区，横栏，这个地区不是很商业，却是休息的好地方。
4. 4号簇有4个镇/区，火炬、东升、民众、三乡，它们不是很商业，而是生活城镇。你可以很容易地找到快餐、中餐、粤菜。
5. 5号簇有5个镇/区，阜沙、南朗、大涌、板芙、神湾，它们有利于旅游业。你可以很容易地找到酒店、花园、餐馆和一些特别的店。

We derives a vertical analysis from the most commonn place informations. We concludes based on the local percentages of each type of venues. The shortage of this analysis is we don't have the horizontal dimension because we don't compare between districts. To have both the Horizontal and Vertical dimensions, a different method will be used: quantify and score each venue in different tags then cluster based on its scores at each tags.

最常见的地点信息导出的是一个垂直分析。我们根据每种场地在当地的比例得出结论。这种分析的不足之处在于我们没有横向维度因为我们没有比较地区之间的差异。为了同时拥有水平和垂直的维度，将使用不同的方法:在不同的标签上，量化每个场地得出评分，然后基于每个标签上的分数进行聚类运算。

## 8. Conclusion 结论
Using Foursquare API and simple machine learning technique, we divide Zhongshan into 5 clusters and find out characteritics of each cluster. Based on this infomation, we could make few recommendations for tourists.

If they are first time to visit in Zhongshan, Cluster #1 , #2 or #5 would be the best for them. These are commercial enough which means that they are full of excitement to enjoy. Cluster #4 might be less exciting but good enough for them who want to know more about real Zhongshan. I am not recommending to visit cluster #3 that are located in outskirt of Zhongshan, however, it would be wonderful experience if they've already visited Zhongshan few times and they want to know more about this city.

利用Foursquare API数据库和机器学习，将中山划分为5个簇，并找出每个簇的特征。根据这些信息，我们可以给游客一些建议。

如果他们是第一次来中山，1号、2号或5号是最好的。这些都是足够商业化的，这意味着他们充满了兴奋享受。4号簇可能没有那么令人兴奋，但对于那些想要更多了解中山真实情况的人来说已经足够了。我不建议去参观位于中山郊区的3号簇，但是，如果他们已经来过中山几次了，想要更多地了解这个城市，那将是一次美妙的经历。